# Table of Contents
 <p><div class="lev1"><a href="#Data-Preparation"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></div><div class="lev1"><a href="#Random-Forest"><span class="toc-item-num">2&nbsp;&nbsp;</span>Random Forest</a></div><div class="lev1"><a href="#Encode-Labels"><span class="toc-item-num">3&nbsp;&nbsp;</span>Encode Labels</a></div><div class="lev1"><a href="#Features-selection"><span class="toc-item-num">4&nbsp;&nbsp;</span>Features selection</a></div><div class="lev1"><a href="#Initialization"><span class="toc-item-num">5&nbsp;&nbsp;</span>Initialization</a></div><div class="lev1"><a href="#Fitting-the-model"><span class="toc-item-num">6&nbsp;&nbsp;</span>Fitting the model</a></div><div class="lev1"><a href="#Predict-on-test-data"><span class="toc-item-num">7&nbsp;&nbsp;</span>Predict on test data</a></div><div class="lev1"><a href="#Prepare-submission"><span class="toc-item-num">8&nbsp;&nbsp;</span>Prepare submission</a></div><div class="lev1"><a href="#Submission"><span class="toc-item-num">9&nbsp;&nbsp;</span>Submission</a></div>

In [31]:
import pandas as pd
import numpy as np

## Data Preparation

In [41]:
%load load_and_enrich.py

CPU times: user 2.91 s, sys: 234 ms, total: 3.14 s
Wall time: 4.03 s
CPU times: user 2.57 s, sys: 157 ms, total: 2.73 s
Wall time: 2.73 s
CPU times: user 4.3 s, sys: 522 ms, total: 4.83 s
Wall time: 4.83 s
CPU times: user 623 ms, sys: 43.3 ms, total: 667 ms
Wall time: 669 ms
CPU times: user 596 ms, sys: 16.2 ms, total: 612 ms
Wall time: 612 ms


## Random Forest

A random forest is an ensemble of decision trees which will output a prediction value. Each decision tree is constructed by using a random subset of the training data. After you have trained your forest, you can then pass each test row through it, in order to output a prediction.

This particular sklearn function requires floats for the input variables, so all strings need to be converted, and any missing data needs to be filled.

## Encode Labels

In [ ]:
%load prepare_data_for_sklearn.py

In [43]:
X.head(3)

,Hour,Month,Year,X,Y,DayOfWeek
0,23,5,2015,-122.425892,37.774599,2
1,23,5,2015,-122.425892,37.774599,2
2,23,5,2015,-122.424363,37.800414,2


## Features selection

In [25]:
features = ['Hour','Month','Year','X','Y','DayOfWeek']

In [26]:
features

['Hour', 'Month', 'Year', 'X', 'Y', 'DayOfWeek']

## Initialization

In [27]:
from sklearn import ensemble
ensemble.RandomForestClassifier?

In [29]:
model = ensemble.RandomForestClassifier(n_jobs=-1,n_estimators=50)

## Fitting the model

In [30]:
%time model.fit(X, y)

CPU times: user 3min 44s, sys: 19.2 s, total: 4min 3s
Wall time: 1min 22s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Predict on test data

In [33]:
%time predictedCategory = model.predict(X_test)

CPU times: user 1min 3s, sys: 1min 10s, total: 2min 14s
Wall time: 1min 8s


In [34]:
predictedCategory

array(['WARRANTS', 'WARRANTS', 'OTHER OFFENSES', ..., 'LARCENY/THEFT',
       'VANDALISM', 'FORGERY/COUNTERFEITING'], dtype=object)

## Prepare submission

In [46]:
def prepareSubmission(df, predictedCategory, sampleDf, category):
    tosubmit = pd.DataFrame({'Id' : df.index.values, '' : predictedCategory})
    tosubmit = pd.get_dummies(tosubmit,prefix_sep='')
    for cat in categories:
        if cat not in tosubmit:
            tosubmit[cat]=0

    tosubmit = tosubmit[sampleDf.columns]
    if len(tosubmit.columns) != (len(categories)+1):
        print('submit data is inconsistent with categories passed')

    return tosubmit

In [47]:
np.unique(predictedCategory)

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype=object)

In [48]:
categories = sorted(train.Category.unique())
%time tosubmit = prepareSubmission(test,predictedCategory,sample,categories)

CPU times: user 1.02 s, sys: 473 ms, total: 1.5 s
Wall time: 1.5 s


In [51]:
tosubmit.head(2)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [52]:
sample.head(2)

,Id,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## Submission

In [160]:
%time tosubmit.to_csv(r'data/tosubmit.csv',index=False)

CPU times: user 28.2 s, sys: 1.29 s, total: 29.5 s
Wall time: 32.3 s
